### Imports

In [1]:
import os
import json
import openai
import pandas as pd

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv('MY_OPENAI')

### Loading Reference Data

In [4]:
df = pd.read_csv("KnowledgeTransfers-one_sent.csv")

In [5]:
### Selecting all annotated cases of family relations
df_fam = df[df.relation.isin(["parent_of", "child_of", "siblings", "spouses"])].copy(deep=True)

In [6]:
df_fam.drop(columns=['annotationspan', 'annotation', 'source', 'attributes'], inplace=True)

### Preparing Data for Entailment Task

In [7]:
df_know = df_fam.reset_index().copy(deep=True)

In [8]:
prop = []
entail = []
for index, row in df_fam.iterrows():
    text = row["relation"]
    person_1 = row["arg1"]
    person_2 = row["arg2"]
    if index % 2 == 0:
        if "parent_of" in text:
            sen = f"{person_1} is the parent of {person_2}."
            prop.append(sen)
            entail.append("entail")
        elif "child_of" in text:
            sen = f"{person_1} is the child of {person_2}."
            prop.append(sen)
            entail.append("entail")
        elif "siblings" in text:
            sen = f"{person_1} and {person_2} are siblings."
            prop.append(sen)
            entail.append("entail")
        elif "spouses" in text:
            sen = f"{person_1} and {person_2} are spouses."
            prop.append(sen)
            entail.append("entail")
    else:
        if "parent_of" in text:
            sen = f"{person_1} is not the parent of {person_2}."
            prop.append(sen)
            entail.append("not entail")
        elif "child_of" in text:
            sen = f"{person_1} is not the child of {person_2}."
            prop.append(sen)
            entail.append("not entail")
        elif "siblings" in text:
            sen = f"{person_1} and {person_2} are spouses."
            prop.append(sen)
            entail.append("not entail")
        elif "spouses" in text:
            sen = f"{person_1} and {person_2} are siblings."
            prop.append(sen)
            entail.append("not entail")

In [9]:
props = pd.Series(prop)
entails = pd.Series (entail)

In [10]:
df_know['proposition'] = props
df_know['classification'] = entails

In [11]:
df_know.classification.value_counts()

classification
entail        50
not entail    39
Name: count, dtype: int64

In [12]:
df_know.head(10)

,index,corpus,drama,context,target,relation,arg1,arg2,proposition,classification
0,16,gdc,11f78.0,Mein Leben sollte Zu freiem Dienste dir gewidm...,audience,parent_of,PARENT 1[IPHIGENIE],iphigenie,PARENT 1[IPHIGENIE] is the parent of iphigenie.,entail
1,18,gdc,11f78.0,Auch hab' ich stets auf dich gehofft und hoffe...,audience,child_of,diana,zeus,diana is the child of zeus.,entail
2,42,gdc,11f78.0,"Das ist's, warum mein blutend Herz nicht heilt...","[audience, arkas]",parent_of,PARENT 2[IPHIGENIE],iphigenie,PARENT 2[IPHIGENIE] is the parent of iphigenie.,entail
3,43,gdc,11f78.0,"In erster Jugend, da sich kaum die Seele An V...","[audience, arkas]",siblings,SIBLINGS[IPHIGENIE],iphigenie,SIBLINGS[IPHIGENIE] and iphigenie are spouses.,not entail
4,226,gdc,11f78.0,"Ruhig herrschte Der König, und es war dem Haus...","[audience, thoas]",siblings,iphigenie,orest,iphigenie and orest are siblings.,entail
5,318,gdc,11f78.0,"Es ist der Weg des Todes, den wir treten: Mit ...",audience,siblings,diana,apollo,diana and apollo are siblings.,entail
6,1189,gdc,11f78.0,"– Ja, vernimm, o König, Es wird ein heimlicher...",thoas,siblings,iphigenie,orest,iphigenie and orest are spouses.,not entail
7,1481,gdc,11f81.0,MARIE. Daß unser Bruder nicht kommt! Es sind z...,audience,siblings,BROTHER[MARIE],"[marie, sophie]","BROTHER[MARIE] and [marie, sophie] are spouses.",not entail
8,1758,gdc,11f81.0,In der Tiefe ihres Jammers schreibt die Ältest...,clavigo,siblings,marie,beaumarchais,marie and beaumarchais are siblings.,entail
9,3273,gdc,11g3h.0,Hast du je Sie denn gefühlt? Verkümmerte dir n...,audience,child_of,CHILD 1[HERZOG],herzog,CHILD 1[HERZOG] is not the child of herzog.,not entail


### Prompt Templates

In [14]:
### With this prompt template, we achieved an F1 score of 0.48 and an accuracy of 0.54

prompt_template = """
  A text T textually entails a proposition P, iff typically, a human would be justified in reasoning from the propositions expressed by T to the proposition expressed by H.
  ###
  Is the {proposition} entailed by the following piece of German {text}?
  Take your time to think.
  Answer with:
  a.) Yes, the proposition is entailed by the given text.
  b.) No, the proposition is not entailed by the given text.
  ###
  Your answer:
  """

In [14]:
### With this prompt template, we achieved an F1 score of 0.52 and an accuracy of 0.57

prompt_template_2 = """
Common sense reasoning exam
###
Explain your reasoning in detail than answer with "Yes, the proposition is entailed by the given text" or "No, the proposition is not entailed by the given text".
Your answer should follow this 4-line format:

Premise: <some sentences from a German play>.
Question: <question requiring logical deduction>.
Reasoning: <an explanation of what you understand about the possible scenarios>.
Answer: <"Yes, the proposition is entailed by the given text" or "No, the proposition is not entailed by the given text">.

###
Premise: German {text}
Question: {proposition}
Reasoning: Let's think logically step by step. T
Answer:
"""

In [15]:
# Function to check entailment using GPT-4
def entail_with_gpt4(text, proposition):
    prompt = ChatPromptTemplate.from_template(template=prompt_template_2)
    messages = prompt.format_messages(text=text, proposition=proposition)
    chat = ChatOpenAI(temperature=0.0, model="gpt-4")
    response = chat(messages)
    #print(response.content)
    res = response.content
    print(res)
    return res

In [16]:
# Inference
results = []
for index, row in df_know.iterrows():
    proposition = row['proposition']
    text = row['context']
    res = entail_with_gpt4(text, proposition)
    results.append(res)

No, the proposition is not entailed by the given text.
he text mentions "Ja, Tochter Zeus'" which translates to "Yes, daughter of Zeus". This phrase is referring to Diana. Therefore, it can be inferred that Diana is the child of Zeus.
Answer: Yes, the proposition is entailed by the given text.
No, the proposition is not entailed by the given text.
No, the proposition is not entailed by the given text.
he text mentions "zwischen beiden Schwestern nun Orest, Der Liebling, wuchs", which translates to "between both sisters now Orest, the favorite, grew up". This implies that Orest is the brother of the two sisters mentioned in the text. However, the text does not provide the names of these sisters. Therefore, we cannot definitively say that Iphigenie is one of these sisters and thus a sibling of Orest.

Answer: No, the proposition is not entailed by the given text.
he text mentions "Im Tempel seiner vielgeliebten Schwester, Die über Tauris herrscht". This translates to "In the temple of hi

In [17]:
pred = []
for i in results:
    if "Yes, the proposition is entailed" in i:
        pred.append("entail")
    else:
        pred.append("not entail")

In [18]:
results_se = pd.Series(pred)

In [19]:
results_se.value_counts()

not entail    77
entail        12
Name: count, dtype: int64

### Evaluation of Output

In [20]:
ground_truth = df_know.classification
predictions = results_se

In [21]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [22]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="macro")],
                      [recall_score(ground_truth, predictions,  average="macro")],
                      [precision_score(ground_truth, predictions, average="macro")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.529755,0.62,0.753247,0.573034
